In [ ]:
from scipy.fft import rfft, fft, fftshift, ifft, fftn, ifftn
from scipy.signal import fftconvolve, kaiser
from scipy import signal
import matplotlib.pyplot as plt
import numpy as np

def freq2time(hf, axis=0):
    return hf.shape[axis] * ifft(hf, axis=axis)

def time2freq(ht, axis=0):
    return fft(ht, axis=axis) / ht.shape[axis]

In [ ]:
import pickle
with open ("reproduce_0/filters_full_0.pkl", "rb") as fh:
    hf_time = pickle.load(fh)
    
wavefield = time2freq(freq2time(hf_time,axis=1),axis=0)
nchan = wavefield.shape[1]
start_chan = nchan*5//8
end_chan = nchan*7//8
noise = wavefield[:,start_chan:end_chan]
print(f'start={start_chan} end={end_chan} size={np.size(noise)}')
variance = np.mean(np.abs(noise)**2)
rms = np.sqrt(variance)
print(f'rms={rms} log10(rms)={np.log10(rms)}')

power=np.abs(fftshift(wavefield))**2

In [ ]:
plotthis = np.log10(power)
plt.imshow(plotthis.T, aspect="auto", origin="lower", cmap="cubehelix_r", vmin=np.log10(rms), interpolation='none')
plt.colorbar()

In [ ]:
plt.imshow(plotthis.T, aspect="auto", origin="lower", cmap="cubehelix_r", vmin=np.log10(rms))
plt.colorbar()

In [ ]:
def smooth_2d_array(array, duty_cycle, beta):
    # Create the Kaiser window
    ashape = np.asarray(array.shape)
    wshape = np.round(ashape * duty_cycle)
    print(f'wshape={wshape}')
    kernel = np.outer(kaiser(wshape[0], beta), kaiser(wshape[1], beta))
    kernel /= np.sum(kernel)  # Normalize the kernel

    # Convolve the array with the kernel
    smoothed_array = fftconvolve(array, kernel, mode='same')

    return smoothed_array

In [ ]:
smpow = smooth_2d_array (power, 0.05, 6)
plotthis = np.log10(smpow)
plt.imshow(plotthis.T, aspect="auto", origin="lower", cmap="cubehelix_r")
plt.colorbar()

In [ ]:
plotthis = np.log10(smpow)
plt.imshow(plotthis.T, aspect="auto", origin="lower", cmap="cubehelix_r", vmin=np.log10(rms/3), interpolation='none')
plt.colorbar()

In [ ]:
def complex_threshold(x: np.ndarray, threshold: float):
    """
    Any value with abs(x) < threshold is set to zero
    """
    out = np.heaviside(np.abs(x) - threshold, 1) * x
    nonz = np.count_nonzero(out)
    sz = np.size(out)
    print(f"complex_threshold: zero={(sz-nonz)*100.0/sz} %")

    return out

In [ ]:
temp=complex_threshold(wavefield, rms)
plotthis = np.log10(np.abs(temp)+rms*1e-4)
plt.imshow(plotthis, aspect="auto", origin="lower", cmap="cubehelix_r", vmin=np.log10(rms), interpolation='none')
plt.colorbar()

In [ ]:
wsum1=np.log10(np.sum(np.abs(wavefield)**2,axis=0))
plt.plot(wsum1)
plt.show()
plt.clf()

In [ ]:
print(wavefield[0,0])

In [ ]:
print(np.max(np.abs(wavefield)))

In [ ]:
wavefield[0,0]=0
wavefield.shape

In [ ]:
hf_time = time2freq(freq2time(wavefield,axis=0),axis=1)

In [ ]:
print(np.max(np.abs(hf_time)))

In [ ]:
hft = np.ones_like(hf_time)

In [ ]:
h = freq2time(time2freq(hft,axis=0),axis=1)

In [ ]:
print(h)

In [ ]:
        h_time_delay = np.zeros_like(hf_time)
        h_time_delay[:,0] = 4096
        h_doppler_delay = fft(h_time_delay, axis=0) / h_time_delay.shape[0]
print(h_doppler_delay)

In [ ]:
0.2176825252584063**2

In [ ]:
0.0473856818028767*4096